AnomalyEvaluation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# import sys
# sys.path.append('/content/drive/MyDrive/AI')

In [ ]:
# from erfnet import ERFNet

In [ ]:
!pip install ood_metrics

In [ ]:
# import erfnet
# !git clone https://github.com/shyam671/AnomalySegmentation_CourseProjectBaseCode.git

Cloning into 'AnomalySegmentation_CourseProjectBaseCode'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 61 (delta 8), reused 8 (delta 8), pack-reused 44
Receiving objects: 100% (61/61), 21.48 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [ ]:
# import zipfile
# import os

# zip_file_path = '/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode.zip'
# extracted_path = os.path.dirname(zip_file_path)

# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extracted_path)

# print(f'File unzipped to: {extracted_path}')

File unzipped to: /content/drive/MyDrive


In [ ]:
! pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=52ec536ee28c194ba09fa1a5213560666d9e5e77493d2d8e4267c409fe8c57fc
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
import sys
import wget
sys.path.append('/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode/eval')
sys.path.append('/content/drive/MyDrive/road-anomaly-benchmark-master/methods')

import baselines
from iouEval import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import wget
sys.path.append('/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode/eval')
sys.path.append('/content/drive/MyDrive/road-anomaly-benchmark-master/methods')

import baselines
from iouEval import *
import eval_iou
from baselines import *
import os
import cv2
import glob
import torch
import random
from PIL import Image
import numpy as np
import erfnet
from erfnet import ERFNet
import os.path as osp
from argparse import ArgumentParser
from ood_metrics import fpr_at_95_tpr, calc_metrics, plot_roc, plot_pr,plot_barcode
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score

seed = 42

# general reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

NUM_CHANNELS = 3
NUM_CLASSES = 20
# gpu training specific
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

def main():
    # parser = ArgumentParser()
    # parser.add_argument(
    #     "--input",
    #     default="/content/drive/MyDrive/AI/Validation_Dataset/RoadObsticle21/images/*.webp",
    #     nargs="+",
    #     help="A list of space separated input images; "
    #     "or a single glob pattern such as 'directory/*.jpg'",
    # )
    # parser.add_argument('--loadDir',default="../trained_models/")
    # parser.add_argument('--loadWeights', default="erfnet_pretrained.pth")
    # parser.add_argument('--loadModel', default="erfnet.py")
    # parser.add_argument('--subset', default="val")  #can be val or train (must have labels)
    # parser.add_argument('--datadir', default="/content/drive/MyDrive/AI/leftImg8bit/val/frankfurt/*.png")
    # parser.add_argument('--num-workers', type=int, default=4)
    # parser.add_argument('--batch-size', type=int, default=1)
    # parser.add_argument('--cpu', action='store_true')
    # args = parser.parse_args(args=[])

    anomaly_score_list = []
    ood_gts_list = []
    means_iou = []
    iou_per_class = []
    input = "/content/drive/MyDrive/Dataset/Validation_Dataset/Validation_Dataset/RoadAnomaly21/images/*.png"
    input_labels_masks = "/content/drive/MyDrive/Dataset/Validation_Dataset/Validation_Dataset/RoadAnomaly21/labels_masks/*.png"
    image_files = glob.glob(input)
    label_files = glob.glob(input_labels_masks)
    loadDir = '/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode/trained_models'
    loadWeights = '/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode/trained_models/erfnet_pretrained.pth'
    loadModel = '/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode/eval/erfnet.py'
    subset = 'val'
    loaddir = ''
    batch_size = 1
    cpu = False

    if not os.path.exists('results.txt'):
        open('results.txt', 'w').close()
    file = open('results.txt', 'a')

    modelpath =  loadModel
    weightspath =  loadWeights

    print("Loading model: " + modelpath)
    print("Loading weights: " + weightspath)

    model = ERFNet(NUM_CLASSES)

    if (not cpu):
      model = torch.nn.DataParallel(model).cuda()

    def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                if name.startswith("module."):
                    own_state[name.split("module.")[-1]].copy_(param)
                else:
                    # print(name, " not loaded")
                    continue
            else:
                own_state[name].copy_(param)
        return model

    model = load_my_state_dict(model, torch.load(weightspath, map_location=lambda storage, loc: storage))
    print("Model and weights LOADED successfully")
    model.eval()

    msp = Max_softmax(model)
    # ent_max = Entropy_max(model)
    # iou = iouEval(NUM_CLASSES)

    # print = (len(label_files), len(image_files))

    for idx, path in enumerate(image_files):
        # print(path)
        label_path = label_files[idx]
        images = torch.from_numpy(np.array(Image.open(path).convert('RGB'))).unsqueeze(0).float()
        images = images.permute(0,3,1,2)
        label_mask = torch.from_numpy(np.array(Image.open(label_path))).unsqueeze(0).float()
        label_mask = label_mask.unsqueeze(0)
        label_mask = label_mask.permute(0,3,1,2)
        with torch.no_grad():
            result = model(images)

        # anomaly_result = msp.anomaly_score(images)       #Max_Softmax_Probability(MSP) score analysis function
        # anomaly_result = ent_max.anomaly_score(images)        #Max_Entropy score analysis function
        anomaly_result = 1.0 - np.max(result.squeeze(0).data.cpu().numpy(), axis=0)     #Max_Logit score analysis function


        # anomaly_score = Max_softmax.anomaly_score(image_np)
        pathGT = path.replace("images", "labels_masks")
        if "RoadObsticle21" in pathGT:
           pathGT = pathGT.replace("webp", "png")
        if "fs_static" in pathGT:
           pathGT = pathGT.replace("jpg", "png")
        if "RoadAnomaly" in pathGT:
           pathGT = pathGT.replace("jpg", "png")

        mask = Image.open(pathGT)
        ood_gts = np.array(mask)

        if "RoadAnomaly" in pathGT:
            ood_gts = np.where((ood_gts==2), 1, ood_gts)
        if "LostAndFound" in pathGT:
            ood_gts = np.where((ood_gts==0), 255, ood_gts)
            ood_gts = np.where((ood_gts==1), 0, ood_gts)
            ood_gts = np.where((ood_gts>1)&(ood_gts<201), 1, ood_gts)

        if "Streethazard" in pathGT:
            ood_gts = np.where((ood_gts==14), 255, ood_gts)
            ood_gts = np.where((ood_gts<20), 0, ood_gts)
            ood_gts = np.where((ood_gts==255), 1, ood_gts)
        ood_gts_ = torch.Tensor(ood_gts).unsqueeze(0).unsqueeze(0)
        anomaly_result_ = torch.Tensor(anomaly_result).unsqueeze(0).unsqueeze(0)

        iou.reset()   #correct
        # print(anomaly_result_.shape,ood_gts_.shape)
        iou.addBatch(anomaly_result_, ood_gts_)  #correct
        # miou, iou_arr = iou.getIoU
        f_iou = iou.getIoU()  # correct

        print(f'mean_IoU in each iteration: {f_iou[0].item()}')
        print(f'IoU_per_class in each iteration: {f_iou[1][0].item()}')

        # means_iou.append(f_iou[0].item())
        # iou_per_class.append(f_iou[1][0].item())

        if 1 not in np.unique(ood_gts):
            continue
        else:
             ood_gts_list.append(ood_gts)
             anomaly_score_list.append(anomaly_result)
        del result, anomaly_result, ood_gts, mask
        torch.cuda.empty_cache()

    # final_iou_mean = sum(means_iou) / len(means_iou)
    file.write( "\n")

    ood_gts = np.array(ood_gts_list)
    anomaly_scores = np.array(anomaly_score_list)

    ood_mask = (ood_gts == 1)
    ind_mask = (ood_gts == 0)

    ood_out = anomaly_scores[ood_mask]
    ind_out = anomaly_scores[ind_mask]

    ood_label = np.ones(len(ood_out))
    ind_label = np.zeros(len(ind_out))

    val_out = np.concatenate((ind_out, ood_out))
    val_label = np.concatenate((ind_label, ood_label))

    prc_auc = average_precision_score(val_label, val_out)
    fpr = fpr_at_95_tpr(val_out, val_label)

    print(f'AUPRC score: {prc_auc*100.0}')
    print(f'FPR@TPR95: {fpr*100.0}')

    file.write(('    AUPRC score:' + str(prc_auc*100.0) + '   FPR@TPR95:' + str(fpr*100.0) ))
    file.close()

    # print(f'mean_IoU for all iteration is: {final_iou_mean}')



main()

Model and weights LOADED successfully
AUPRC score: 9.600823675910778
FPR@TPR95: 93.78032503061641


In [ ]:
# !python /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input '/content/drive/MyDrive/AI/Validation_Dataset/FS_LostFound_full/images/*.png' --loadDir ./